In [3]:
import os
import pandas as pd
import numpy as np
from ast import literal_eval
%matplotlib inline
import matplotlib.pyplot as plt
from shutil import copyfile

In [4]:
folder = os.path.join("regioni.old.4","torch_sir_validation_grid_search")
best_path = os.path.join(os.getcwd(), folder, "best")
if not os.path.exists(best_path):
    os.mkdir(best_path)

regions = ["Sardegna", "P.A. Bolzano"]
for region in regions:
    path = os.path.join(folder, region, "scores.csv")
    df = pd.read_csv(path, sep="\t")
    no_alpha_df = df[(df.use_alpha == False)]
    best_exp_name = no_alpha_df[no_alpha_df.val_risk == no_alpha_df.val_risk.min()].name.values[0]
    print(no_alpha_df[no_alpha_df.name == best_exp_name].val_risk)
    print(no_alpha_df[no_alpha_df.name == best_exp_name].test_risk)
    print(no_alpha_df[no_alpha_df.name == best_exp_name].dataset_risk)

    print(best_exp_name)
#     suffixs = ["_bcd_over_time", "_I_fit", "_W_fit", "_SIR_global", "_r0"]
#     file_type = ".png"
#     for suffix in suffixs:
#         copyfile(os.path.join(os.getcwd(), folder, region, best_exp_name + suffix + file_type), 
#                  os.path.join(folder, "best", best_exp_name + suffix + file_type))


59    7.816322
Name: val_risk, dtype: float64
59    45.925426
Name: test_risk, dtype: float64
59    14.110578
Name: dataset_risk, dtype: float64
S_b0.8_g0.3_d0.02_lrb0.0001_lrg1e-05_lrd3e-06_lra0.001_ts45_vl10_st_der100000.0_nd_der0_t_inc0.1_use_alphaFalse_y_loss_weight0.0
58    3.808903
Name: val_risk, dtype: float64
58    246.77562
Name: test_risk, dtype: float64
58    91.88786
Name: dataset_risk, dtype: float64
P_b0.8_g0.3_d0.02_lrb0.0001_lrg1e-05_lrd3e-06_lra0.001_ts45_vl10_st_der20000.0_nd_der0_t_inc0.1_use_alphaFalse_y_loss_weight0.0


In [5]:
df = pd.read_csv(os.path.join("scores.csv"), sep="\t")
# df[df.lr_gamma <= 0.000003].min()
df[df.val_risk == df.val_risk.min()]

FileNotFoundError: [Errno 2] File scores.csv does not exist: 'scores.csv'

In [ ]:
# df.groupby(["beta_t0","train_size"]).dataset_risk.mean()

In [ ]:
def compute_r0(df, t):
    df["r0_t" + str(t)] = df["final_beta_t" + str(t)] / df["final_gamma_t" + str(t)]
    
def get_list_from_column(df, raw_pos, column, max_size):
    col_val = df[column].iloc[raw_pos]
    x = literal_eval(col_val)
    return x +  [-1.0]*(max_size - len(x))

def get_res_parameters(scores_df, params=("beta", "gamma", "delta")):
    max_len = scores_df["train_size"].max()
    p_columns = ["final_" + p + "_t" + str(i) for p in params for i in range(max_len)]
    score_columns = list(set(scores_df.columns) - set(params))
    #     params_df.at[:, score_columns] = scores_df.loc[:, score_columns] # copies everything in the first columns
    
    rows = []
    for i in range(scores_df.shape[0]):
        # create rows
        values = []
        for p in params:
            val = get_list_from_column(scores_df, i, p, max_len)
            values.extend(val)
        
        rows.append(list(scores_df.iloc[i][score_columns]) + values)
        
    
    params_df = pd.DataFrame(rows, columns=score_columns + p_columns)
    for p in p_columns:
        params_df.loc[params_df[p] == -1, p] = None

    for t in range(max_len):
        compute_r0(params_df, t)
    
    return params_df, p_columns
        
df = pd.read_csv(path, sep="\t")        
params_df, p_columns = get_res_parameters(df)


In [ ]:
# params_df[[p_columns[0], p_columns[-1]]].plot.box()
params_df[["final_beta_t0", "final_gamma_t0", "final_delta_t0"]].plot.box(title=region +" initial estimated params")
params_df[["final_beta_t39", "final_gamma_t39", "final_delta_t39"]].plot.box(title=region + " final estimated params")
params_df[["r0_t0", "r0_t39"]].plot.box(title=region +" estimated initial and final R0")

In [ ]:
# params_df.loc[[0], ["r0_t"+ str(t) for t in range(20)]].plot()
params_df.groupby(["beta_t0","train_size"]).dataset_risk.mean().plot(title=region + " dataset risk with different beta, train_size", legend=True)

In [ ]:
fig, ax = plt.subplots()
r0_cols = ["r0_t"+ str(t) for t in range(39)]
ax.plot([1.0]*len(r0_cols), '-r')
params_df[r0_cols].mean().plot(ax=ax, title=region + " averaged R0 over time")

In [ ]:
# R0 of single experiment
exp_id = params_df.val_risk.idxmin()
print(params_df.iloc[exp_id]["name"])
r0_cols = ["r0_t"+ str(t) for t in range(39)]
fig, ax = plt.subplots()
ax.plot([1.0]*len(r0_cols), '-r')
params_df[r0_cols].iloc[exp_id].plot(ax=ax, label="$r_0$", legend=True, title=region + " R0 over time")
plt.savefig(os.path.join(region, "r0.pdf"))

In [ ]:
params_df.dataset_risk.idxmin()

In [ ]:
# best_res = params_df[params_df["dataset_risk"] < 120]
best_res = params_df[params_df["dataset_risk"] < 30]
best_res[["final_beta_t0", "final_gamma_t0", "final_delta_t0"]].plot.box(title=region +" initial estimated params")
best_res[["final_beta_t39", "final_gamma_t39", "final_delta_t39"]].plot.box(title=region + " final estimated params")
best_res[["r0_t0", "r0_t39"]].plot.box(title=region +" estimated initial and final R0")

In [ ]:
params_df.iloc[17]["final_delta_t0"]